In [1]:
%load_ext autoreload
%autoreload 2
from preprocess import Preprocessor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

p_train = Preprocessor('train')
p_test = Preprocessor('test')

load course date info over!
('number of courses:', 39)
load course object info over!
('number of courses:', 39)
('number of moduels:', 26750)
load course object info over!
('size of log data:', 4677908)
('number of enrollment_ids:', 72395)
load enrollment info over!
('number of courses:', 39)
('number of enrollments:', 72395)
('number of uers: ', 53870)
load truth info over!
('number of labels:', 72395)
load course date info over!
('number of courses:', 39)
load course object info over!
('number of courses:', 39)
('number of moduels:', 26750)
load course object info over!
('size of log data:', 1548480)
('number of enrollment_ids:', 24013)
load enrollment info over!
('number of courses:', 39)
('number of enrollments:', 24013)
('number of uers: ', 21266)
load truth info over!
('number of labels:', 24013)


## Load features

In [7]:
df = p_train.get_features_all()

event_count features extracted! 1.456823 seconds taken
('Shape of the event_count dataframe: ', (72395, 8))
weekly_session_count features extracted! 76.777606 seconds taken
('Shape of the weekly_session_count dataframe: ', (72395, 6))
problem_video_ratio features extracted! 3.273508 seconds taken
('Shape of the problem_video dataframe: ', (72395, 3))
All features extracted! 81.665883 seconds taken
('Shape of the whole features dataframe: ', (72395, 16))


In [8]:
df.head()

,enrollment_id,access_count,discussion_count,navigate_count,page_close_count,problem_count,video_count,wiki_count,week_one_session,week_two_session,week_three_session,week_four_session,week_five_session,week_six_session,problem_ratio,video_ratio,label
0,1,107,0,25,66,80,29,0,1,5,8,3,3,0,0.476190,0.446429,0
1,4,61,0,15,8,6,4,0,5,7,6,0,0,0,0.047619,0.053571,0
2,5,226,34,30,87,152,86,0,6,25,2,19,5,0,0.450549,0.327434,0
3,7,194,33,19,59,84,69,0,5,4,7,4,0,0,0.296703,0.230088,1
4,13,198,4,15,25,116,69,0,0,4,0,13,12,0,0.126214,0.219355,0


In [9]:
df.shape

(72395, 17)

## Deal with outliers

In [19]:
## Using the formula to calculate the outlier
## Using Quantile(0.75)+1.5IQR as upper cut-off line (As Boxplot)
## Using Quantile(0.25)-1.5IQR as lower cut-off line (As Boxplot)
## Transform the outliers to the nearest cut-off line
num_feature = list(df.columns[1:-1])
for item in num_feature:
    Q3 = df[item].quantile(0.75)
    Q1 = df[item].quantile(0.25)
    IQR = Q3-Q1
    upper = Q3+1.5*IQR
    lower = Q1-1.5*IQR
    df.loc[df[item] > upper, item] = upper
    df.loc[df[item] < lower, item] = lower
df.shape

(72395, 17)

## Deal with low variance feature

In [10]:
## Dropping variables where 95% or more of the values are the same
## Find the ratio of the second most frequent value to the most frequent value for each predictor 
## Remove variables where this ratio was less than 0.05
num_feature = list(df.columns[1:-1])
for value in num_feature:
    freq_set = df[value].value_counts()
    max_1 = freq_set.max()
    max_2 = 0
    for num in freq_set:
        if num == max_1:
            continue
        max_2 = num
        break
    ratio = max_2/float(max_1)
    print value,":",ratio
    #if ratio < 0.05:
        #df = df.drop(value,1)      

access_count : 0.369840270224
discussion_count : 0.117887706116
navigate_count : 0.872527472527
page_close_count : 0.501494733937
problem_count : 0.0370471261237
video_count : 0.432727544686
wiki_count : 0.197316790613
week_one_session : 0.131963839483
week_two_session : 0.204539122608
week_three_session : 0.164960974833
week_four_session : 0.141752727132
week_five_session : 0.103509143543
week_six_session : 0.00950075302265
problem_ratio : 0.0393370209718
video_ratio : 0.0727343194884


In [23]:
df.shape

(72395, 14)

## Data transformation

In [24]:
df.head()

,enrollment_id,access_count,discussion_count,navigate_count,page_close_count,problem_count,video_count,wiki_count,week_two_session,week_three_session,week_four_session,problem_ratio,video_ratio,label
0,1,61.0,0.0,22.0,25.0,12.5,15.0,0.0,2.5,2.5,2.5,0.04717,0.090909,0
1,4,61.0,0.0,15.0,8.0,6.0,4.0,0.0,2.5,2.5,0.0,0.04717,0.053571,0
2,5,61.0,5.0,22.0,25.0,12.5,15.0,0.0,2.5,2.0,2.5,0.04717,0.090909,0
3,7,61.0,5.0,19.0,25.0,12.5,15.0,0.0,2.5,2.5,2.5,0.04717,0.090909,1
4,13,61.0,4.0,15.0,25.0,12.5,15.0,0.0,2.5,0.0,2.5,0.04717,0.090909,0


- Normalization Data

In [86]:
num_feature = list(df.columns[1:-1])
for item in num_feature:
    df[item]=(df[item]-df[item].min())/(df[item].max()-df[item].min())

In [87]:
df.head()

,enrollment_id,access_count,discussion_count,navigate_count,page_close_count,problem_count,video_count,wiki_count,week_one_event,week_two_event,week_three_event,week_four_event,week_two_session,week_three_session,week_four_session,label
0,1,1.0,0.0,1.000000,1.00,1.0,1.000000,0.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,0
1,4,1.0,0.0,0.681818,0.32,0.5,0.266667,0.0,1.0,1.0,1.000000,0.0,1.0,1.0,0.0,0
2,5,1.0,1.0,1.000000,1.00,1.0,1.000000,0.0,1.0,1.0,0.470588,1.0,1.0,1.0,1.0,0
3,7,1.0,1.0,0.863636,1.00,1.0,1.000000,0.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1
4,13,1.0,0.8,0.681818,1.00,1.0,1.000000,0.0,0.0,1.0,0.000000,1.0,1.0,0.0,1.0,0
